### PyTorch and HuggingFace scavenger hunt!
PyTorch and HuggingFace have emerged as powerful tools for developing and deploying neural networks.

In this scavenger hunt, we will explore the capabilities of PyTorch and HuggingFace, uncovering hidden treasures on the way.

We have two parts:

- Familiarize  with PyTorch
- Get HuggingFace

### PyTorch tensors

Scan through the PyTorch tensors documentation [here](https://pytorch.org/docs/stable/tensors.html). Be sure to look at the examples.

In the following cell, create a tensor named `my_tensor` of size 3x3 with values of your choice. The tensor should be created on the GPU if available. Print the tensor.

In [1]:
from transformers import BertTokenizer as bt
from transformers import BertForSequenceClassification as bf
import torch
import torch.nn as nn
import torch.optim as optim

### Neural Net Constructor Kit `torch.nn`

You can think of the `torch.nn` ([documentation](https://pytorch.org/docs/stable/nn.html)) module as a constructor kit for neural networks. It provides the building blocks for creating neural networks, including layers, activation functions, loss functions, and more.

Instructions:

Create a three layer Multi-Layer Perceptron (MLP) neural network with the following specifications:

- Input layer: 784 neurons
- Hidden layer: 128 neurons
- Output layer: 10 neurons

Use the ReLU activation function for the hidden layer and the softmax activation function for the output layer. Print the neural network.

Hint: MLP's use "fully-connected" or "dense" layers. In PyTorch's `nn` module, this type of layer has a different name. See the examples in [this tutorial](https://pytorch.org/tutorials/recipes/recipes/defining_a_neural_network.html) to find out more.

In [2]:
class MyMLP(nn.Module):
    """My simple MLP model
    Specifications:
    - input layer: 784 neurons
    - hidden layer: 128 neurons with ReLU activation
    - output layer: 10 neurons with softmax activation
    """

    def __init__(self):
        super(MyMLP, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        # pass thee input to the second layer
        x = torch.flatten(x, start_dim=1)
        # apply the ReLU activation function
        x = torch.relu(self.fc1(x))
        # pass the result to the final layer
        x = self.fc2(x)
        # apply the softmax activation function
        x = self.softmax(x)
        return x

In [3]:
my_mlp = MyMLP()
print(my_mlp)

MyMLP(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
  (relu): ReLU()
  (softmax): Softmax(dim=1)
)


In [4]:
# Check the number of inputs
assert my_mlp.fc1.in_features == 784

# Check the number of outputs
assert my_mlp.fc2.out_features == 10

# Check the number of nodes in the hidden layer
assert my_mlp.fc1.out_features == 128

# Check that my_mlp.fc1 is a fully connected layer
assert isinstance(my_mlp.fc1, nn.Linear)

# Check that my_mlp.fc2 is a fully connected layer
assert isinstance(my_mlp.fc2, nn.Linear)

### PyTorch Loss Functions and Optimizers

PyTorch comes with a number of built-in loss functions and optimizers that can be used to train neural networks. The loss functions are implemented in the `torch.nn` ([documentation](https://pytorch.org/docs/stable/nn.html#loss-functions)) module, while the optimizers are implemented in the `torch.optim` ([documentation](https://pytorch.org/docs/stable/optim.html)) module.


Instructions:

- Create a loss function using the `torch.nn.CrossEntropyLoss` ([documentation](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)) class.
- Create an optimizer using the `torch.optim.SGD` ([documentation](https://pytorch.org/docs/stable/generated/torch.optim.SGD.html#torch.optim.SGD)) class with a learning rate of 0.01.



In [5]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(my_mlp.parameters(), lr=0.01)

In [9]:
# Check

assert isinstance(
    loss_function, nn.CrossEntropyLoss
), "loss_fn should be an instance of CrossEntropyLoss"
assert isinstance(optimizer, torch.optim.SGD), "optimizer should be an instance of SGD"
assert optimizer.defaults["lr"] == 0.01, "learning rate should be 0.01"
assert optimizer.param_groups[0]["params"] == list(
    my_mlp.parameters()
), "optimizer should be passed the MLP parameters"

### PyTorch Training Loops
PyTorch makes writing a training loop easy!


In [10]:
def fake_training_loaders():
    for _ in range(30):
        yield torch.randn(64, 784), torch.randint(0, 10, (64,))


for epoch in range(3):
    # Create a training loop
    for i, data in enumerate(fake_training_loaders()):
        # Every data instance is an input + label pair
        x, y = data

        # Zero your gradients for every batch!
        optimizer.zero_grad()

        # Forward pass (predictions)
        y_pred = my_mlp(x)

        # Compute the loss and its gradients
        loss = loss_function(y_pred, y)
        loss.backward()

        # Adjust learning weights
        optimizer.step()

        if i % 10 == 0:
            print(f"Epoch {epoch}, batch {i}: {loss.item():.5f}")

Epoch 0, batch 0: 2.30199
Epoch 0, batch 10: 2.30316
Epoch 0, batch 20: 2.30135
Epoch 1, batch 0: 2.30354
Epoch 1, batch 10: 2.29946
Epoch 1, batch 20: 2.30431
Epoch 2, batch 0: 2.30483
Epoch 2, batch 10: 2.30576
Epoch 2, batch 20: 2.30669


In [12]:
# Check

assert abs(loss.item() - 2.3) < 0.1, "the loss should be around 2.3 with random data"

## HuggingFace

### Download a model from HuggingFace and use it for sentiment analysis

HuggingFace provides a number of pre-trained models that can be used for a variety of tasks. In this exercise, we will use the `distilbert-base-uncased-finetuned-sst-2-english` model to perform sentiment analysis on a movie review.

Instructions:
- Review the [AutoModel tutorial](https://huggingface.co/docs/transformers/quicktour#automodel) on the HuggingFace website.
- Instantiate an AutoModelForSequenceClassification model using the `distilbert-base-uncased-finetuned-sst-2-english` model.
- Instantiate an AutoTokenizer using the `distilbert-base-uncased-finetuned-sst-2-english` model.
- Define a function that will get a prediction

In [13]:
 # Get the model and tokenizer

from transformers import AutoModelForSequenceClassification, AutoTokenizer

pt_model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")


def get_prediction(review):
    """Given a review, return the predicted sentiment"""

    # Tokenize the review
    # (Get the response as tensors and not as a list)
    inputs = tokenizer(review, return_tensors="pt")

    # Perform the prediction (get the logits)
    outputs = pt_model(**inputs)

    # Get the predicted class (corresponding to the highest logit)
    predictions = torch.argmax(outputs.logits, dim=-1)

    return "positive" if predictions.item() == 1 else "negative"

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [14]:
# Check

review = "This movie is not so great :("

print(f"Review: {review}")
print(f"Sentiment: {get_prediction(review)}")

assert get_prediction(review) == "negative", "The prediction should be negative"


review = "This movie rocks!"

print(f"Review: {review}")
print(f"Sentiment: {get_prediction(review)}")

assert get_prediction(review) == "positive", "The prediction should be positive"

Review: This movie is not so great :(
Sentiment: negative
Review: This movie rocks!
Sentiment: positive


### Download a dataset from HuggingFace

HuggingFace provides a number of datasets that can be used for a variety of tasks. In this exercise, we will use the `imdb` dataset and pass it to the model we instantiated in the previous exercise.

Instructions:
- Review the [loading a dataset](https://huggingface.co/docs/datasets/v1.11.0/loading_datasets.html) documentation
- Fill in the blanks

In [15]:

from datasets import load_dataset

# Load the test split of the imdb dataset
dataset = load_dataset("imdb", split="test")

dataset

README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [16]:
# Check

from pprint import pprint

from datasets import Dataset

assert isinstance(dataset, Dataset), "The dataset should be a Dataset object"
assert set(dataset.features.keys()) == {
    "label",
    "text",
}, "The dataset should have a label and a text feature"

# Show the first example
pprint(dataset[0])

{'label': 0,
 'text': 'I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV '
         'are usually underfunded, under-appreciated and misunderstood. I '
         'tried to like this, I really did, but it is to good TV sci-fi as '
         'Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap '
         "cardboard sets, stilted dialogues, CG that doesn't match the "
         'background, and painfully one-dimensional characters cannot be '
         "overcome with a 'sci-fi' setting. (I'm sure there are those of you "
         "out there who think Babylon 5 is good sci-fi TV. It's not. It's "
         'clichéd and uninspiring.) While US viewers might like emotion and '
         'character development, sci-fi is a genre that does not take itself '
         'seriously (cf. Star Trek). It may treat important issues, yet not as '
         "a serious philosophy. It's really difficult to care about the "
         'characters here as they are not simply foolish, ju

### Now let's use the pre-trained model!

Let's make some predictions.

Instructions:
- Fill in the blanks

In [17]:
# Get the last 3 reviews
reviews = dataset["text"][-3:]

# Get the last 3 labels
labels = dataset["label"][-3:]

# Check
for review, label in zip(reviews, labels):
    # Let's use your get_prediction function to get the sentiment
    # of the review!
    prediction = get_prediction(review)

    print(f"Review: {review[:80]} \n... {review[-80:]}")
    print(f'Label: {"positive" if label else "negative"}')
    print(f"Prediction: {prediction}\n")

Review: I got Monster Man in a box set of three films where I mainly wanted the other tw 
... ous, often gnarly splatter comedy that should endear itself to fans of the same.
Label: positive
Prediction: positive

Review: Five minutes in, i started to feel how naff this was looking, you've got a compl 
... for anyone who likes their horror with several side orders of gore and attitude.
Label: positive
Prediction: positive

Review: I caught this movie on the Sci-Fi channel recently. It actually turned out to be 
... e more than passable for the horror/slasher buff. Definitely worth checking out.
Label: positive
Prediction: positive

